Hi Mike, this is a markdown cell

you need to install the Jupyter from microsoft extension in Visual Studio Code (easiest) from the left panel - plugins  


In [2]:
1+2+3+4

10

In [10]:
# TODO: import needed modules

from lxml import etree
import math as m
import numpy as np
import pandas as pd
import attr
import collections 
import pathlib as pl

File Directories can be easily copied with the following procedure:
1) open windows files manager and go to directory
2) on the directory press and hold the shift key and left click the mouse and select - "Copy as path"
3) key press "control v" below

In [18]:
# TODO: input QC dir
inputDir=input("Input Dir Path to Be Checked")
inputDir=pl.Path(inputDir)
# print(inputDir)
inputDir

WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place')

In [23]:
# TODO: check dir for h2k files
def get_files(extensions):
    all_files = {}
    for ext in extensions:
        
        all_files[ext]=[f.name for f in (inputDir.glob(ext))]
        # all_files.extend(pl.Path('.').glob(ext))
    return all_files

files = get_files(('*.h2k', '*.jpg', '*.pdf'))
files

{'*.h2k': ['5EADD00001.h2k', '5EXXD0003.h2k'],
 '*.jpg': ['5EADD00001 - Datasheet.jpg',
  '5EXXD0003-ATTIC ACCESS.jpg',
  '5EXXD0003-BACK.jpg',
  '5EXXD0003-BASEBOARD.jpg',
  '5EXXD0003-DHW.jpg',
  '5EXXD0003-FP.jpg',
  '5EXXD0003-FRONT.jpg',
  '5EXXD0003-LEFT.jpg',
  '5EXXD0003-LEFT_2.jpg',
  '5EXXD0003-RIGHT.jpg'],
 '*.pdf': ['5EXXD00003-DATASHEET.pdf',
  '5EXXD0003HOIS(EN).pdf',
  '5EXXD0003LBL(EN).pdf',
  '5EXXD0003SRUR(EN).pdf']}

In [ ]:
# TODO: read h2k xml file

# TODO: display basic customer data




In [ ]:
# TODO: check files in  directory names for extra spaces an jpg extentions...

In [2]:
# TODO: check all reqired files are uploaded